# Leven at Newby Bridge using HBV model in eWaterCycle
This notebook demonstrates how eWaterCycle is used to run a model, in this case the classic HBV model, with forcing from the Caravan dataset.

For this example, we chose a catchment in the Lake District in the UK. River discharge in river Leven is measured at the weirs at Newby Bridge, see the photo below. This observational data is available through the CamelsGB dataset [(Coxon, 2020)](https://essd.copernicus.org/articles/12/2459/2020/).
The larger [Caravan](https://www.nature.com/articles/s41597-023-01975-w) (collection of Camels...) dataset is available through eWaterCycle and will be used below.

![image](https://upload.wikimedia.org/wikipedia/commons/7/76/Weirs_on_the_River_Leven_at_Newby_Bridge_-_geograph.org.uk_-_5455774.jpg)

*Weirs on the River Leven at Newby Bridge by G Laird*

As a model we choose the classic [HBV model](https://hess.copernicus.org/articles/26/1371/2022/) for its simplicity. This nicely demonstrates how eWaterCycle works. More complex models are available through eWaterCycle, however:

- these often require parameter sets specific to a region
- these are more computationally intensive to run and therefore unsuited for a short 45 minute workshop.

Ask us about available models if you want to collaborate! 

### HBV model structure

The stucture of the HBV model is shown below.
Do note that the figure is missing a snow reservoir, however this has been implemented in the HBV model in this notebook.

![image](_images/model_layout.png)

*Image from the TU Delft course ENVM1502 - "River Basin Hydrology" by Markus Hrachowitz* 


## Starting up
To start up, we need to import eWaterCycle and a number of general Python libraries. 'Under the hood' eWaterCycle depends on a large number of other pieces of software, including but not limited to

- [grpc4bmi](https://github.com/eWaterCycle/grpc4bmi), a 'translator' for BMI function calls between different programming languages and across containers. This library was build by the eWaterCycle team, but is available openly for anyone that can benefit from its functionality. 
- apptainer, a container engine that runs the model-containers (Docker is supported too).
- [ESMValTool](https://github.com/ESMValGroup/ESMValTool), a climate data processing toolbox that originally intended to post-process climate data from CMIP projects for inclusion in IPCC reports, we adopted as tool for pre-processing climate data into forcing data for hydrological models
- Numerous hydrological models that are made available as plugins to eWaterCycle, see [eWaterCycle-leakybucket](https://github.com/eWaterCycle/ewatercycle-leakybucket) as an example. Note that plugins do not have to be owned and maintained by the eWaterCycle team: anyone with a model can make a plugin for eWaterCycle and make their model be available for others through the platform. 

Furthermore, eWaterCycle requires forcing data, obsrvational data and parameter sets to be available to users. If you want to install eWaterCycle on your own infrastructure, see the [eWaterCycle documentation](https://ewatercycle.readthedocs.io/en/latest/system_setup.html) or contact us.

In [7]:
# general python
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

import numpy as np
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr

#niceties
from rich import print

In [8]:
# general eWaterCycle
import ewatercycle
import ewatercycle.models
import ewatercycle.forcing

## Choose region and time period: 
The HBV model is a lumped hydrological model. It considers a catchment as a homogenious area and calculates the major hydrological processes in it. It requires spatially aggregated rainfall and potential evaporation as inputs (ie. forcings). To calculate its modelled discharge at the outlet of the catchment it also needs a set of parameters. Usually these paramters are calibrated using (historical) observational data, so this is also required. 

in eWaterCycle we provide access to the Caravan dataset, which contains all of the above data for all the catchments in the different Camels datasets. In this notebook we use the precipitation and evaporation data from Caravan. However, there is a known problem with the caravan evaporation data and (the current version) shouldn't be used. In this notebook we do so to demonstrate how it works. It would be better to generate forcing from (for example) ERA5. [This](example_model_run_HBV_camels_catchment_ERA5_forcing.ipynb) additional notebook that looks very similar to this one explains how to do that. 

Using the interactive maps at [eWaterCycle caravan map](https://www.ewatercycle.org/caravan-map/) one can easily retrieve the identifier of the catchment.

Note that changing the region below will work, but that the parameters that are loaded later in this notebook are calibrated specifically for this particular catchment!

In [9]:
camelsgb_id = "lamah_208082"

We have to specify start and end date of the experiment that we want to do. For now we don't fuzz with diverences between calibration and validation periods (which officially of course is very bad...)

In [10]:
experiment_start_date = "1979-08-01T00:00:00Z"
experiment_end_date = "2023-08-31T00:00:00Z"

## Set up paths

Since forcing files are often re-used between experiments it is a best practice to save those intermediate files for re-use between experiments. These logical save-points in workflows are called 'rustpunten' in Dutch. It is important to store data in 'rustpunten' in standard formats. Working with clearly defined 'rustpunten' is a key element in the design of good workflows in general and was instrumental in designing eWaterCycle in particular. 

Here we set up some paths to store the forcing files we generate in your own home directory. 

To speed up this workshop, we have already created the forcing files in a central location, which we also create pointers to here. If you want to run for a different region, you will have to generate the forcing yourself. 

In [11]:
# Even though we don't use caravan data as forcing, we still call it forcing
# because it is generated using the forcing module of eWaterCycle
forcing_path_caravan = Path.home() / "forcing" / camelsgb_id / "caravan"
forcing_path_caravan.mkdir(exist_ok=True, parents=True)

# If someone has prepared forcing for you, this path needs to be changed to that location. 
prepared_forcing_path_caravan_central = Path("location/of/forcing/data")

## Generate or load forcing
There are three options for creating forcing data objects:

- generate from climate data such as Caravan, ERA5 or CMIP. Note that if the directory you specify as destination already contains data this trying this will throw an error!
- load forcing data you generated previously by providing the location where it was stored
- load forcing data someone else (such as your teacher or a workshop leader) generated previously by providing the location where it was stored

First we will create a caravan forcing object, but as mentioned above, we will only use this for the discharge observations and the shape file of the region. After generating the object we show the fields it contains and plot the discharge data. 

The actual forcing object we will use in this example is the ERA5 based data.

For both caravan and ERA5 based forcing, only one of the options provided below should be used, use comments to select which one you want to use.

In [12]:
# option one: generate forcing data
camelsgb_forcing = ewatercycle.forcing.sources['CaravanForcing'].generate(
    start_time=experiment_start_date,
    end_time=experiment_end_date,
    directory=forcing_path_caravan,
    basin_id=camelsgb_id,
)

syntax error, unexpected WORD_WORD, expecting SCAN_ATTR or SCAN_DATASET or SCAN_ERROR
context: <!DOCTYPE^ html PUBLIC "-//W3C//DTD XHTML 1.1//EN" "http://www.w3.org/TR/xhtml11/DTD/xhtml11.dtd"><html xmlns="http://www.w3.org/1999/xhtml" xml:lang="en"> <head> <title>The page is temporarily unavailable</title> <meta http-equiv="Content-Type" content="text/html; charset=UTF-8" /> <style type="text/css"> /*<![CDATA[*/ body { background-color: #fff; color: #000; font-size: 0.9em; font-family: sans-serif,helvetica; margin: 0; padding: 0; } :link { color: #c00; } :visited { color: #c00; } a:hover { color: #f50; } h1 { text-align: center; margin: 0; padding: 0.6em 2em 0.4em; background-color: #900; color: #fff; font-weight: normal; font-size: 1.75em; border-bottom: 2px solid #000; } h1 strong { font-weight: bold; font-size: 1.5em; } h2 { text-align: center; background-color: #900; font-size: 1.1em; font-weight: bold; color: #fff; margin: 0; padding: 0.5em; border-bottom: 2px solid #000; } h3 { 

OSError: [Errno -70] NetCDF: DAP server error: 'https://opendap.4tu.nl/thredds/dodsC/data2/djht/ca13056c-c347-4a27-b320-930c2a4dd207/1/lamah.nc'

In [ ]:
# # option two or three: load data that you or someone else generated previously
# camelsgb_forcing = ewatercycle.forcing.sources['CaravanForcing'].load(directory=prepared_forcing_path_caravan_central)

In [ ]:
print(camelsgb_forcing)

In [ ]:
#quick plot of the discharge data. 
ds_forcing = xr.open_mfdataset([camelsgb_forcing['Q'],camelsgb_forcing['pr'],camelsgb_forcing['evspsblpot']])
ds_forcing["Q"].plot()

#Load the forcing dataset
ds = xr.open_dataset("/home/thirza/forcing/lamah_208082/caravan/lamah_208082_1930-08-01_2024-08-31_pr.nc")

catchment_area = ds["area"].values

# go from mm/day to m3/s
discharge = []
for i in range(len(ds_forcing["Q"].values)):
    discharge.append((ds_forcing["Q"].values[i] * catchment_area * 1000) / (24 * 60 * 60))


In [ ]:
x = ds["time"].values
y = discharge

plt.plot(x,y)
plt.xlabel('time')
plt.ylabel('observed streaflow [$m^3$/s]')

In [ ]:
max(discharge)

In [ ]:
# waardes en aantal van deze waardes boven een drempel bepalen
print([x for x in discharge if x >= 20])
print(len([x for x in discharge if x >= 20]))

In [ ]:
import seaborn as sns
sns.set()

In [ ]:
#see how the Q values are distributed
plt.hist(discharge)

In [ ]:
#see how high Q values are distributed
plt.hist([value for value in discharge if value > 20], bins=30)

In [ ]:
# Find the maximum annual discharge
max_discharge = ds_forcing["Q"].groupby("time.year").max()
maxdischarge = max_discharge.values * catchment_area * 1000 / (24 * 60 * 60) #convert to m3/s
# print(maxdischarge)

In [ ]:
import scipy.stats as stats
import math
# # discharge data > 20 m³/s
# data = [value for value in discharge if value > 20]

# all discharge data
cleaned_discharge = [x for x in discharge if not math.isnan(x)]
data = cleaned_discharge

# maximum annual discharge
# data = [x for x in maxdischarge if not math.isnan(x)]
# # print(data)

# Generalized Extreme Value (GEV) distribution 
shape, loc, scale = stats.genextreme.fit(data)

# Define threshold value
threshold = 530

# calculate exceedance probability in years
p = 1 - stats.genextreme.cdf(threshold, shape, loc=loc, scale=scale)

# calculate return period in years
T = 1 / p / 365.25
# print(T)

# calculate change threshold exceedance at least once in 1000 and 1100 years
P_1000 = 1 - (1 - p) ** 1000
P_1100 = 1 - (1 - p) ** 1100

# Results 1000
print(f"Returnperiod for 530 m³/s: {T:.2f} years")
print(f"Chance of at least one exceedance in 1000 years: {P_1000:.5f} ({P_1000 * 100:.2f}%)")
print(f"Chance of at least one exceedance in 1100 years: {P_1100:.5f} ({P_1100 * 100:.2f}%)")

# expected amount of times threshold is exceeded in 1000 years
expected_exceedances1000 = 1000 / T if T > 0 else 0
print(f"The discharge currently exceeds the threshold of 530 m³/s {expected_exceedances1000:.3f} times in 1000 years")

# expected amount threshold is exceeded in 1100 years
expected_exceedances1100 = 1100 / T if T > 0 else 0
print(f"The discharge currently exceeds the threshold of 530 m³/s {expected_exceedances1100:.3f} times in 1100 years")

In [ ]:
import scipy.stats as stats
import math
# # discharge data > 20 m³/s
# data = [value for value in discharge if value > 20]

# all discharge data
cleaned_discharge = [x for x in discharge if not math.isnan(x)]
data = cleaned_discharge

# maximum annual discharge
# data = [x for x in maxdischarge if not math.isnan(x)]
# # print(data)

# Generalized Extreme Value (GEV) distribution 
shape, loc, scale = stats.genextreme.fit(data)

# Define threshold value
threshold = np.linspace(1, 600, 600)

return_period = []
prob = []
for i in range(len(threshold)):
# calculate exceedance probability in years
    p = 1 - stats.genextreme.cdf(threshold[i], shape, loc=loc, scale=scale)
    prob.append(p)
# calculate return period in years
    T = 1 / p / 365.25
    return_period.append(T)
# print(return_period)

plt.plot(return_period, threshold)

In [ ]:
ds_forcing["pr"].plot(label = 'precipitation')
ds_forcing["evspsblpot"].plot(label = 'potential evaporation')
plt.legend()

Below we generate forcing from ERA5. Note that ERA5 data needs to be stored on the system you are working on. For SURF Research Cloud machines running the eWaterCycle catalog item, this is handled by the eWaterCycle team. 

## Load parameters from calibration

The HBV model contains five "stores" where the water is stored and nine parameters that control the flow between those stores and in and out of the model.
We have already calibrated the model for our region of choice and hardcoded those below. If you have changed the region and have calibrated for your region, you need to point to your calibration results here by uncommenting the load statement and pointing to the right file.

In [ ]:
# load pre-calibrated constants. If you have calibrated for a different region, comment this part and 
# uncomment the load statement below
#par_0 = [7.085, 0.837, 76.373, 1.112, 0.245, 7.801, 0.096, 0.003, 0.226];

par_0 = [ 35.66528028,   1.1858021,  170.34055787,   1.46199007,   1.91737973,
   1.45432159,   0.57091109,   1.88674579,   0.83621774]


# #load calibration constants from a csv file
# par_0 = np.loadtxt("your/calibration/file.csv", delimiter = ",")

In [ ]:


param_names = ["Imax", "Ce", "Sumax", "Beta", "Pmax", "Tlag", "Kf", "Ks", "FM"]
print(list(zip(param_names, np.round(par_0, decimals=3))))

For the storages we can specify an array of starting values. If you don't the model will start 'empty' and needs some timesteps to 'fill up'. Especially for the rootzone storage it helps to not start empty. Note that all units are in mm: <br>

In [ ]:
#               Si,  Su, Sf, Ss, Sp
s_0 = np.array([0,  100,  0,  5,  0])

## Setting up the model
Below we show the core use of eWaterCycle: to users models are objects. We stay as close as possible to the standard BMI functions, but make sure that under the hood eWaterCycle makes everything run. Getting a model to run requires three steps:

1. creating a model object, an instance of the specific model class. This is provided by the different plugins. At the point of creation, the forcing object that will be used need to be passed to the model:

In [ ]:
model = ewatercycle.models.HBVLocal(forcing=camelsgb_forcing)

2. creating a configuration file that contains all the information the model needs to initialize itself.
   The format of the configuration file is very model specific. For example, the HBV configuration file contains information on:
  - the location of forcing files
  - the values of parameters and initial storages

In [ ]:
config_file, _ = model.setup(parameters=par_0, initial_storage=s_0)

3. initializing starts the model container, creates all variables, and basically gets the model primed to start running.

In [ ]:
model.initialize(config_file)

## Running the model
The basic loop that runs the model calls the ```model.update()``` to have the model progress one timestep and ```model.get_value()``` to extract information of interest. More complex experiment can interact with the model using, for example, ```model.set_value()``` to change values. In this way 

- multiple models can interact (including be coupled)
- models can be adjusted during runtime to incoming observations (ie. data assimilation)
- Effects not covered by the model can be calculated seperatly and included to create 'what if' experiments.
- and many more applications.

In [ ]:
Q_m = []
time = []
while model.time < model.end_time:
    model.update()
    Q_m.append(model.get_value("Q")[0])
    time.append(pd.Timestamp(model.time_as_datetime))

After running the model we need to call ```model.finalize()``` to shut everything down, including the container. If we don't do this, the container will continue to be active, eating up system memory.

In [ ]:
model.finalize()

## Process results
Finally, we use standard python libraries to visualize the results. We put the model output into a pandas Series to make plotting easier.

In [ ]:
model_output = pd.Series(data=Q_m, name="Modelled_discharge", index=time)

In [ ]:

ds_forcing["Q"].plot(label="Observed discharge")
model_output.plot()
plt.legend()
plt.ylabel("Discharge (mm/d)")

In [ ]:
model_output.plot()

In [ ]:
#we want to also be able to use the output of this model run in different analyses. Therefore we save it as a NetCDF file
xr_model_output = model_output.to_xarray()

xr_model_output.attrs['units'] = 'mm/d'

# Save the xarray Dataset to a NetCDF file
xr_model_output.to_netcdf('~/river_discharge_data.nc')